In [1]:
import pandas as pd

In [2]:
trip = pd.read_csv("tripadvisor_hotel_reviews.csv")

In [3]:
trip.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [4]:
trip = trip[(trip[ 'Rating']==5) | (trip[ 'Rating'] ==2) | (trip[ 'Rating' ]==1)][[ 'Review', 'Rating']]  
trip['Rating'] = trip['Rating'].apply(lambda rating: 'Pos' if rating==5 else 'Neg')


In [5]:
trip.reset_index(inplace=True) 
trip.head()

,index,Review,Rating
0,1,ok nothing special charge diamond member hilto...,Neg
1,3,"unique, great stay, wonderful time hotel monac...",Pos
2,4,"great stay great stay, went seahawk game aweso...",Pos
3,5,love monaco staff husband stayed hotel crazy w...,Pos
4,6,"cozy stay rainy city, husband spent 7 nights m...",Pos


In [6]:
trip['Rating'].value_counts()


Pos    9054
Neg    3214
Name: Rating, dtype: int64

In [7]:
import re
import nltk

In [8]:
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer 
from nltk.stem import WordNetLemmatizer

In [9]:
import nltk
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [10]:
nltk.download('stopwords') 
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [12]:
ps = WordNetLemmatizer()
corpus = []

In [13]:
for i in range(0, len(trip)):
    review = re.sub('[^a-zA-Z]'," ",trip['Review'][i])
    review = review.lower()
    review = review.split()
    review = [ps.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [14]:
trip.to_csv('tdata.csv',index=False)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [16]:
x = cv.fit_transform(corpus).toarray()

In [17]:
x.shape

(12268, 34569)

In [18]:
y = pd.get_dummies(trip['Rating'])
y = y.iloc[:,1].values

In [19]:
y

array([0, 1, 1, ..., 0, 0, 0], dtype=uint8)

In [20]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=3)

In [21]:
from sklearn.naive_bayes import MultinomialNB 
model1 = MultinomialNB().fit(X_train, y_train)
model1.score(X_test,y_test)

0.9539527302363489

In [22]:
y_pred = model1.predict(X_test)

In [23]:
from sklearn.metrics import confusion_matrix 
confusion_m = confusion_matrix(y_test,y_pred)

In [24]:
confusion_m

array([[ 574,   69],
       [  44, 1767]], dtype=int64)

In [25]:
from sklearn.metrics import accuracy_score 
accuracy = accuracy_score(y_test,y_pred)

In [26]:
accuracy

0.9539527302363489

In [27]:
from sklearn.metrics import precision_score 
precision_score(y_test,y_pred)

0.9624183006535948

In [28]:
from sklearn.metrics import recall_score
recall_score(y_test,y_pred)

0.9757040309221424

In [29]:
from sklearn.metrics import fbeta_score 
fbeta_score(y_test, y_pred,beta=1)

0.9690156292843433

In [30]:
positive_message = "the hotel was nice and comfy"
negative_message = "the hotel was bad and the staff was rude"
data = [negative_message]
vect = cv.transform(data).toarray()
my_prediction = model1.predict(vect)
my_prediction_prob = model1.predict_proba(vect)
if my_prediction==1: 
    print("Positive") 
    print(my_prediction_prob[0][1])

else:

    print("Negative")
    print (my_prediction_prob[0][0])


Negative
0.8445454685022524


In [31]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver = 'liblinear')

In [32]:
model = logreg.fit(X_train,y_train)

In [33]:
y_pred_class = logreg.predict(X_test)

In [34]:
from sklearn import metrics
metrics.roc_auc_score (y_test, y_pred)

0.9341972720707136

In [35]:
metrics.confusion_matrix (y_test,y_pred)

array([[ 574,   69],
       [  44, 1767]], dtype=int64)

In [36]:
positive_message = "cozy stay rainy city, wonderful time at the hotel. "
negative_message = "The first thing that you notice about this hotel is the stench of m. "
data= [positive_message.lower()]
vect=cv.transform(data).toarray()
my_prediction=model.predict(vect)

if my_prediction <= 1:
    print("Positive") 
    print(my_prediction_prob[0][1])
else:
    print("Negative") 
    print(my_prediction_prob[0][0])

Positive
0.15545453149774663


In [37]:
positive_message = "the hotel was nice and comfy" 
negative_message = "the hotel was bad and the staff was rude"
data = [negative_message.lower()]
my_prediction = model.predict(vect)
my_prediction_prob = model.predict_proba(vect)

if my_prediction==1:
    print("Positive") 
    print (my_prediction_prob[0][1])

else:

    print("Negative")
    print (my_prediction_prob[0][0])

Positive
0.9046083528511962


In [38]:
import pickle

filename = 'logistic_regression_model.pk1'
pickle.dump(model, open(filename, 'wb')) 
pickle.dump(cv, open('transform_logistic.pkl', 'wb'))

In [39]:
pickle.dump(cv, open('tranform.pk1', 'wb'))
filename = 'naive_bayes_model.pkl' 
pickle.dump(model, open(filename, 'wb'))